# **NEW APPROACH WITH IDENTIFY* ***

In [39]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import pandas as pd
import numpy as np
import os
from sklearn.metrics.pairwise import cosine_similarity
import matplotlib.pyplot as plt
import pickle
import os

# Load plant details dataset
df = pd.read_excel("/kaggle/input/sci-data/sci.xlsx")  # Update with your file path

# Set dataset path
data_dir = "/kaggle/input/med-images/images"  # Update with your dataset path
img_size = (224, 224)
batch_size = 32


In [40]:
datagen = ImageDataGenerator(
    rescale=1.0/255,
    validation_split=0.2,  # 80% train, 20% validation
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.3,
    horizontal_flip=True
)

# Load training and validation data
train_data = datagen.flow_from_directory(
    data_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical',
    subset='training'
)
val_data = datagen.flow_from_directory(
    data_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation'
)

num_classes = len(train_data.class_indices)
print(f"Number of classes: {num_classes}")

# Get the number of classes
num_classes = len(train_data.class_indices)
print(f"Number of classes: {num_classes}")

# Save class indices to a .pkl file
class_indices = train_data.class_indices
pkl_path = "/kaggle/working/class_indices.pkl"
with open(pkl_path, 'wb') as f:
    pickle.dump(class_indices, f)
print(f"class_indices saved to {pkl_path}")

Found 8088 images belonging to 100 classes.
Found 1978 images belonging to 100 classes.
Number of classes: 100
Number of classes: 100
class_indices saved to /kaggle/working/class_indices.pkl


In [3]:
# Load pre-trained InceptionV3 model
base_model = keras.applications.InceptionV3(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freeze initial layers but allow fine-tuning on the top layers
base_model.trainable = True
for layer in base_model.layers[:200]:  
    layer.trainable = False

# Define the new model
model = keras.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(256, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.3),
    layers.Dense(num_classes, activation='softmax')
])

# Compile model with Adam optimizer
optimizer = keras.optimizers.Adam(learning_rate=0.0001)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

model.summary()



87910968/87910968 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ inception_v3 (Functional)            │ (None, 5, 5, 2048)          │      21,802,784 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ global_average_pooling2d             │ (None, 2048)                │               0 │
│ (GlobalAveragePooling2D)             │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 256)                 │         524,544 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 256)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 128)                 │          32,896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 100)                 │          12,900 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 22,373,124 (85.35 MB)

 Trainable params: 15,374,628 (58.65 MB)

 Non-trainable params: 6,998,496 (26.70 MB)

In [4]:
# Set number of epochs
epochs = 50  # Adjust as needed

# Train the model
history = model.fit(train_data, validation_data=val_data, epochs=epochs)

# Save trained model
model.save("classifier_inceptionv3.h5")


/usr/local/lib/python3.10/dist-packages/PIL/Image.py:1054: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


Epoch 1/50


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


253/253 ━━━━━━━━━━━━━━━━━━━━ 359s 1s/step - accuracy: 0.0128 - loss: 4.6405 - val_accuracy: 0.0546 - val_loss: 4.5062
Epoch 2/50
253/253 ━━━━━━━━━━━━━━━━━━━━ 208s 802ms/step - accuracy: 0.0442 - loss: 4.4625 - val_accuracy: 0.1259 - val_loss: 4.1492
Epoch 3/50
253/253 ━━━━━━━━━━━━━━━━━━━━ 208s 803ms/step - accuracy: 0.1124 - loss: 4.1101 - val_accuracy: 0.2068 - val_loss: 3.8236
Epoch 4/50
253/253 ━━━━━━━━━━━━━━━━━━━━ 206s 797ms/step - accuracy: 0.1689 - loss: 3.8116 - val_accuracy: 0.2513 - val_loss: 3.5202
Epoch 5/50
253/253 ━━━━━━━━━━━━━━━━━━━━ 207s 799ms/step - accuracy: 0.2252 - loss: 3.5144 - val_accuracy: 0.2998 - val_loss: 3.2369
Epoch 6/50
253/253 ━━━━━━━━━━━━━━━━━━━━ 206s 797ms/step - accuracy: 0.2714 - loss: 3.2845 - val_accuracy: 0.3296 - val_loss: 3.0467
Epoch 7/50
253/253 ━━━━━━━━━━━━━━━━━━━━ 206s 797ms/step - accuracy: 0.3405 - loss: 2.9257 - val_accuracy: 0.3680 - val_loss: 2.8881
Epoch 8/50
253/253 ━━━━━━━━━━━━━━━━━━━━ 206s 799ms/step - accuracy: 0.3682 - loss: 2.7471 

In [13]:
# Load the saved Inception model
model = tf.keras.models.load_model("/kaggle/working/classifier_inceptionv3.h5")

In [14]:
def load_and_preprocess_image(image_path, target_size=(224, 224)):
    # Load image
    img = image.load_img(image_path, target_size=target_size)
    
    # Convert to array
    img_array = image.img_to_array(img)
    
    # Expand dimensions to match the input shape of the model (1, 224, 224, 3)
    img_array = np.expand_dims(img_array, axis=0)
    
    # Preprocess the image (normalization)
    img_array = preprocess_input(img_array)
    
    return img_array


In [15]:
def predict_plant(image_path, model, class_indices, df):
    # Preprocess the input image
    img_array = load_and_preprocess_image(image_path)
    
    # Make predictions
    predictions = model.predict(img_array)
    
    # Get the predicted class index
    predicted_class_idx = np.argmax(predictions, axis=1)[0]
    
    # Get the predicted plant name based on class index
    predicted_plant_name = list(class_indices.keys())[list(class_indices.values()).index(predicted_class_idx)]
    
    # Retrieve plant details from the dataset (df)
    plant_details = df[df['Scientific_name'] == predicted_plant_name].to_dict(orient='records')[0]
    
    return predicted_plant_name, plant_details


In [16]:
# Load plant dataset containing scientific names and details (df)
df = pd.read_excel("/kaggle/input/sci-data/sci.xlsx")  # Update with your dataset file path

In [24]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.inception_v3 import preprocess_input
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os

# Load the saved Inception model
model = tf.keras.models.load_model("/kaggle/working/classifier_inceptionv3.h5")

# Load the plant dataset
df = pd.read_excel("/kaggle/input/sci-data/sci.xlsx")  # Update with your dataset file path

# Define dataset path and parameters
data_dir = "/kaggle/input/med-images/images"  # Update with your image dataset path
img_size = (224, 224)
batch_size = 32

# Create the ImageDataGenerator for loading the images
datagen = ImageDataGenerator(
    rescale=1.0/255,
    validation_split=0.2  # Assuming you split the data earlier
)

# Load training data to get class_indices
train_data = datagen.flow_from_directory(
    data_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical',
    subset='training'  # Load the training subset
)

# Retrieve class_indices
class_indices = train_data.class_indices

# Function to preprocess the image
def load_and_preprocess_image(image_path, target_size=(224, 224)):
    img = image.load_img(image_path, target_size=target_size)  # Ensure the 'image' module is imported
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array = preprocess_input(img_array)
    return img_array

# Predict function
def predict_plant(image_path, model, class_indices, df):
    img_array = load_and_preprocess_image(image_path)
    predictions = model.predict(img_array)
    predicted_class_idx = np.argmax(predictions, axis=1)[0]
    predicted_plant_name = list(class_indices.keys())[list(class_indices.values()).index(predicted_class_idx)]
    plant_details = df[df['Scientific_name'] == predicted_plant_name].to_dict(orient='records')[0]
    return predicted_plant_name, plant_details



Found 8088 images belonging to 100 classes.


In [35]:
# Input image path
image_path = "/kaggle/input/med-images/images/Celastrus paniculatus/Image_30.jpg"  # Update with your test image path

# Predict
plant_name, plant_details = predict_plant(image_path, model, class_indices, df)

# Output
print(f"Predicted Plant: {plant_name}")
print(f"Plant Details: {plant_details}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
Predicted Plant: Celastrus paniculatus
Plant Details: {'Scientific_name': 'Celastrus paniculatus', 'Telugu_name': 'Pengu', 'Parts_used': 'Leaves, Seeds, Oil, Bark', 'Uses': 'Nervine Tonic', 'Grown_Area': 'Natural Source in Forest Area of above\n500 mtr. Height', 'Preparation_method': "The seeds are diluted in cow's water and applied to sores on the gums. Powder the seeds and give 1 spoonful 2-3 times a day to relieve cough, constipation and flatulence.", 'Preparation_method(Telugu)': 'వల్లరై, తులసి ఆకులు, మిరియాలపొడిని తీసుకుని మిరప పొడిలా గ్రైండ్ చేసుకోవాలి. అన్ని సూరాలు చుట్టి రెండుసార్లు ఇస్తాయి. ఆకులను గ్రైండ్ చేయడం వల్ల ఏనుగు పాదాల వాపు, దద్దుర్లు, కణితుల నుంచి ఉపశమనం లభిస్తుంది. మెంతి ఆకులతో కషాయం తయారుచేసి పిల్లలకు 1 నుంచి 2 కోన్లు ఇస్తే గుండెల్లో మంట, కడుపు సమస్యలు నయం అవుతాయి. ఆవు పాలలో ఆకు పొడిని కలపడం వల్ల జ్ఞాపకశక్తి పెరుగుతుంది. రోజూ నాలుగు ఆకులు తింటే జ్ఞానం పెరుగుతుంది. వల్లరై రుతుస్రావాన్ని తీవ్రతరం చేసే (అబార్షన్) ప్రభావాన్ని కలిగి

In [41]:
import requests
from PIL import Image
from io import BytesIO
import numpy as np
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.inception_v3 import preprocess_input

# Function to download and preprocess image from URL
def download_image_from_url(url, target_size=(224, 224)):
    response = requests.get(url)  # Download the image
    img = Image.open(BytesIO(response.content))  # Open the image
    img = img.resize(target_size)  # Resize to match model input
    img_array = image.img_to_array(img)  # Convert to array
    img_array = np.expand_dims(img_array, axis=0)  # Expand dimensions to match model input
    img_array = preprocess_input(img_array)  # Preprocess the image
    return img_array

# Predict function for URL-based image
def predict_plant_from_url(image_url, model, class_indices, df):
    img_array = download_image_from_url(image_url)  # Download and preprocess image from URL
    predictions = model.predict(img_array)  # Predict the class
    predicted_class_idx = np.argmax(predictions, axis=1)[0]
    predicted_plant_name = list(class_indices.keys())[list(class_indices.values()).index(predicted_class_idx)]
    plant_details = df[df['Scientific_name'] == predicted_plant_name].to_dict(orient='records')[0]
    return predicted_plant_name, plant_details

# Example: Predict from a URL
image_url = "https://bloomingfloret.in/cdn/shop/files/6091-2_1.jpg?v=1725259539"  # Replace with the actual URL of the image
plant_name_url, plant_details_url = predict_plant_from_url(image_url, model, class_indices, df)

# Output
print(f"Predicted Plant (URL): {plant_name_url}")
print(f"Plant Details (URL): {plant_details_url}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
Predicted Plant (URL): Aloe vera
Plant Details (URL): {'Scientific_name': 'Aloe vera', 'Telugu_name': 'Kalabanda', 'Parts_used': 'Leaf, Root', 'Uses': ' Tonic, Emmenagogue, Refrigerant', 'Grown_Area': 'All over Andhra Pradhesh', 'Preparation_method': 'Kumari (a) Rice cactus: Take the blood and rind after cutting the young lobe with the skin and adding cumin seeds.\nMixed discharge and body heat are cured. Wash the rice of the aloe several times, add some Chinese sugar to it and let the water drain in it to hang it in small pieces. If this is done in the eyes, conjunctivitis etc. will become. Its leaves can be used to cure chronic ulcers. Take its rice, oil it and rub it on the head to induce hair growth all over the head. Applying its milk on the eyes cures sores in the eyeballs. Take its rice and apply it on your face to reduce acne. Take 3 teaspoons of its rice daily and grind it with the required amount of native sugar and make the juice in the 